<a href="https://colab.research.google.com/github/ykenkou001/icevision_memo/blob/main/train_pillar_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check GPU
!nvidia-smi

In [ ]:
# # Torch - Torchvision - IceVision - IceData - MMDetection - YOLOv5 - EfficientDet Installation
# !wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh

# # Choose your installation target: cuda11 or cuda10 or cpu
# !bash icevision_install.sh cuda11

In [ ]:
# Torch - Torchvision - IceVision - IceData - MMDetection - YOLOv5 - EfficientDet Installation
!wget https://raw.githubusercontent.com/airctic/icevision/master/icevision_install.sh

# Choose your installation target: cuda11 or cuda10 or cpu
!bash icevision_install.sh cuda11 master

In [ ]:
# Restart kernel after installation、このカーネルを実行すると後々エラーが出る
import IPython
RESTART_KERNEL = False
if RESTART_KERNEL:
  IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
# fiftyoneをソースからリインストール
INSTALL_51 = False
if INSTALL_51:
  import shutil
  shutil.copy('/content/drive/MyDrive/Colab Notebooks/ice_vision/reinstall_51.sh',
              '/content/')
  !sh reinstall_51.sh

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Imports

In [ ]:
from icevision.all import *

In [ ]:
import numpy as np
from pathlib import Path

# Parse  pillar detectiondataset

In [ ]:
# 保存した train_records, valid_recordsを読み込む
import pickle

with open('/content/drive/MyDrive/Colab Notebooks/ice_vision/train/records.pickle', 'rb') as f:
    train_records, valid_records = pickle.load(f)


In [ ]:
# define the parameters and create the parser
annotation_filepath = '/content/drive/MyDrive/Colab Notebooks/ice_vision/train/result_annotation.json'
img_dir = '/content/drive/MyDrive/Colab Notebooks/ice_vision/train/images'

parser = parsers.COCOBBoxParser(annotations_filepath=annotation_filepath, img_dir=img_dir)

In [ ]:
# Parse annotations to create records
# recordsの内容をpickleで保存する場合, cache_filepath='save_path'
# reference -> https://github.com/airctic/icevision/blob/9c17564b82b2c17367ee2eb02fd24dbf8bf6376c/icevision/parsers/parser.py#L94

CREATE_RECORDS = False
if CREATE_RECORDS:
  train_records, valid_records = parser.parse(cache_filepath='/content/drive/MyDrive/Colab Notebooks/ice_vision/train/records.pickle')
parser.class_map

In [ ]:
train_records[3]

# Create datasets with augumentations and transforms

In [ ]:
# Transforms
# size is set to 384 because EfficientDet requires its inputs to be divisible by 128
image_size = 384
train_tfms = tfms.A.Adapter(
    [*tfms.A.aug_tfms(size=image_size, presize=384), tfms.A.Normalize()]) # presize: データ拡張サイズ
valid_tfms = tfms.A.Adapter([*tfms.A.resize_and_pad(image_size), tfms.A.Normalize()])

In [ ]:
# Datasets
train_ds = Dataset(train_records, train_tfms)
valid_ds = Dataset(valid_records, valid_tfms)

In [ ]:
# Show an element of the train_ds with augmentation transformations applied
samples = [train_ds[0] for _ in range(3)]
show_samples(samples, ncols=3)

# Creating a model

## Select a library, model, and backbone

In order to create a model, we need to:
* Choose one of the **libraries** supported by IceVision
* Choose one of the **models** supported by the library
* Choose one of the **backbones** corresponding to a chosen model

You can access any supported models by following the IceVision unified API, use code completion to explore the available models for each library.



In [ ]:
import icevision
icevision.__version__

In [ ]:
# Just change the value of selection to try another model

selection = 13

extra_args = {}

if selection == 0:
  model_type = models.mmdet.vfnet
  backbone = model_type.backbones.resnet50_fpn_mstrain_2x

if selection == 1:
  model_type = models.mmdet.retinanet
  backbone = model_type.backbones.resnet101_fpn_2x
  # backbone = model_type.backbones.resnet50_fpn_1x

  # extra_args['cfg_options'] = { 
  #   'model.bbox_head.loss_bbox.loss_weight': 2,
  #   'model.bbox_head.loss_cls.loss_weight': 0.8,
  #    }

if selection == 2:
  model_type = models.mmdet.faster_rcnn
  backbone = model_type.backbones.resnet101_fpn_2x
  # extra_args['cfg_options'] = { 
  #   'model.roi_head.bbox_head.loss_bbox.loss_weight': 2,
  #   'model.roi_head.bbox_head.loss_cls.loss_weight': 0.8,
  #    }

if selection == 3:
  model_type = models.mmdet.ssd
  backbone = model_type.backbones.ssd300

if selection == 4:
  model_type = models.mmdet.yolox
  backbone = model_type.backbones.yolox_s_8x8

if selection == 5:
  model_type = models.mmdet.yolof
  backbone = model_type.backbones.yolof_r50_c5_8x8_1x_coco

if selection == 6:
  # mmdet/models/dense_heads/detr_head.py: fix 526lines
  # labels[pos_inds] = gt_labels[sampling_result.pos_assigned_gt_inds].to(torch.int64)
  model_type = models.mmdet.detr
  backbone = model_type.backbones.r50_8x2_150e_coco

if selection == 7:
  model_type = models.mmdet.deformable_detr
  backbone = model_type.backbones.twostage_refine_r50_16x2_50e_coco

if selection == 8:
  model_type = models.mmdet.fsaf
  backbone = model_type.backbones.x101_64x4d_fpn_1x_coco

if selection == 9:
  model_type = models.mmdet.sabl
  backbone = model_type.backbones.r101_fpn_gn_2x_ms_640_800_coco

if selection == 10:
  model_type = models.mmdet.centripetalnet
  backbone = model_type.backbones.hourglass104_mstest_16x6_210e_coco

elif selection == 11:
  # The Retinanet model is also implemented in the torchvision library
  model_type = models.torchvision.retinanet
  backbone = model_type.backbones.resnet50_fpn

elif selection == 12:
  model_type = models.ross.efficientdet
  backbone = model_type.backbones.tf_d4
  # The efficientdet model requires an img_size parameter
  extra_args['img_size'] = image_size

elif selection == 13:
  model_type = models.ultralytics.yolov5
  backbone = model_type.backbones.medium
  # The yolov5 model requires an img_size parameter
  extra_args['img_size'] = image_size

model_type, backbone, extra_args

In [ ]:
backbone.__dict__

In [ ]:
parser.class_map

In [ ]:
# Instantiate the model
model = model_type.model(backbone=backbone(pretrained=True), 
                         num_classes=len(parser.class_map), **extra_args)

## データローダ. 
データローダはmodel_typeに固有のものです。データローダの仕事は、データセットからアイテムを取得し、各モデルが必要とする特定のフォーマットでバッチ処理することです。このため、データローダーの作成はデータセットの作成とは別に行われます。  
  
valid_dlからアイテムの最初のバッチを見てみましょう。valid_tfmsはリサイズ(パディングあり)と正規化されたレコードのみを扱うので、例えば毎回異なる画像が返されるわけではないことを覚えておいてください。これは、トレーニング中に一貫した検証を行うために重要です。

In [ ]:
# Data Loaders
train_dl = model_type.train_dl(train_ds, batch_size=8, num_workers=4, shuffle=True)
valid_dl = model_type.valid_dl(valid_ds, batch_size=8, num_workers=4, shuffle=False)

In [ ]:
# show batch
model_type.show_batch(first(valid_dl), ncols=4)

## Metrics

In [ ]:
metrics = [COCOMetric(metric_type=COCOMetricType.bbox)]

## Training 

In [ ]:
# Start tensorboard.
%load_ext tensorboard
%tensorboard --logdir '/content/drive/MyDrive/Colab Notebooks/ice_vision/lightning_logs/'

In [ ]:
class LightModel(model_type.lightning.ModelAdapter):
    def configure_optimizers(self):
        return Adam(self.parameters(), lr=1e-4)

light_model = LightModel(model, metrics=metrics)

In [ ]:
# change directiory
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/ice_vision')

In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger


# model_type = models.ross.efficientdet
# backbone = model_type.backbones.tf_d4

model_name = 'ross.efficientdet'
backbone_name = 'tf_d4'

val_loss = ['ultralytics.yolov5']
valid_slash_loss = ["mmdet.faster_rcnn", "mmdet.detr", "mmdet.retinanet", 
                    "ross.efficientdet"]

if model_name in val_loss:
  mnt = "val_loss"
  aimn = True
  filename = f"{model_name}-{backbone_name}-" + "{epoch:02d}-{val_loss:.4f}"
elif model_name in valid_slash_loss:
  mnt = "valid/loss"
  aimn = False
  filename=f"{model_name}-{backbone_name}-" + "epoch={epoch:02d}-val_loss={valid/loss:.4f}"

early_stop_callback = EarlyStopping(
    monitor=mnt, 
    min_delta=1e-05,
    patience=5,
    verbose=True,
    mode="min")

checkpoint_callback = ModelCheckpoint(
    monitor=mnt,
    dirpath="/content/drive/MyDrive/Colab Notebooks/ice_vision/lightning_logs",
    #　Is useful to set it to ``False`` when metric names contain ``/``as this will result in extra folders.
    auto_insert_metric_name=aimn,
    filename=filename,
    save_top_k=1,
    mode="min",
    verbose=True
)
# https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.loggers.tensorboard.html
logger = TensorBoardLogger(
    save_dir = "/content/drive/MyDrive/Colab Notebooks/ice_vision/lightning_logs",
    name = model_name + '_' + backbone_name,
    version = None
)

In [ ]:
trainer = pl.Trainer(
    # max_epochs=100, gpus=1, callbacks=[checkpoint_callback])
    max_epochs=100, gpus=1, callbacks=[early_stop_callback, checkpoint_callback],
    logger=logger)
trainer.fit(light_model, train_dl, valid_dl)

In [ ]:
from icevision.models import *
# Save The Model
checkpoint_path = f'/content/drive/MyDrive/Colab Notebooks/ice_vision/checkpoints/{model_name}_{backbone_name}.pth'

save_icevision_checkpoint(model, 
                        model_name=model_name, 
                        backbone_name=backbone_name,
                        classes =  parser.class_map.get_classes(), 
                        img_size=image_size, 
                        filename=checkpoint_path,
                        meta={'icevision_version': '12.0.0'})

In [ ]:
infer_dl = model_type.infer_dl(valid_ds, batch_size=4, shuffle=False)
preds = model_type.predict_from_dl(model, infer_dl, keep_images=True)

In [ ]:
show_preds(preds=preds[:4])